In [1]:
import os

In [2]:
computation = 'cloud'

### cloud Import

In [3]:
  # !pip install pyglib
  # !pip install torch
  # !pip install scikit-learn
  # !pip install torch-sparse
  # !pip install torchvision
  # !pip install torch-scatter
  # !pip install torch-geometric
  # !pip install munch
  # !pip install wandb
  # !pip install lz4
  # !pip install zstandard
  # !pip install zstd
  # !pip install tabulate

In [4]:
!pip install pyglib
!pip install torch
!pip install torchvision
import torch
print(f'torch installed with version { torch.__version__ }')

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.1/52.1 kB 5.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyglib: filename=pyglib-0.1-py2.py3-none-any.whl size=4532 sha256=722c662dde8a3aa22ac8e6d2e9e02d8beb9b71d90c60fc1635102ad79b09beca
  Stored in directory: /root/.cache/pip/wheels/0d/c4/e6/bdb4a219df3f70db47bf78b23a61d1d2c5edd2275403f71046
  Created wheel for python-gflags: filename=python_gflags-3.1.2-py3-none-any.whl size=57371 sha256=d3fc95f9d2eb99d9aeea1ddfcad702bb1d776460761d3bfa011aebb61a319389
  Stored in directory: /root/.cache/pip/wheels/08/32/87/be6254803d81af495c135b8b662d4a076e7a91dc7766f05a25
Successfully built pyglib python-gflags
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 107.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 61.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [5]:
!pip install torch-scatter -f https://pytorch-geometric.com/whl/torch-2.6.0+cu124.html
!pip install torch-sparse -f https://pytorch-geometric.com/whl/torch-2.6.0+cu124.html

Looking in links: https://pytorch-geometric.com/whl/torch-2.6.0+cu124.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.8/10.8 MB 57.9 MB/s eta 0:00:00
Looking in links: https://pytorch-geometric.com/whl/torch-2.6.0+cu124.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.0/5.0 MB 53.9 MB/s eta 0:00:00


In [6]:
!pip install torch-geometric

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.1/63.1 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 36.5 MB/s eta 0:00:00


In [7]:
!pip install scikit-learn
!pip install munch
!pip install wandb
!pip install lz4
!pip install zstandard
!pip install zstd
!pip install tabulate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 33.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 44.9 MB/s eta 0:00:00


### ..

In [8]:
if computation=='local' :
    data_path = '../data/AML'
    model_load_path = '../models/GNN'
    model_save_path = '../models/GNN'
else :
  from google.colab import drive
  drive.mount('/content/drive')
  data_path = '/content/drive/MyDrive/uco_fraud_detector/data/AML'
  model_load_path = '/content/drive/MyDrive/uco_fraud_detector/models/GNN'
  model_save_path = '/content/drive/MyDrive/uco_fraud_detector/models/GNN'


print('Searching for file : ','found' if os.path.exists(data_path) else 'not found')
print(os.listdir(data_path))

Mounted at /content/drive
Searching for file :  found
['small']


In [9]:
import os
import pandas as pd
import numpy as np
from datetime import datetime
import itertools
import logging
from sklearn.preprocessing import LabelEncoder
import torch
from torch_geometric.data import Data, HeteroData
from torch_geometric.typing import OptTensor
import torch.nn as nn
from torch_geometric.nn import GINEConv, BatchNorm, Linear, GATConv, PNAConv, RGCNConv
import torch.nn.functional as F
import tqdm
from torch_geometric.transforms import BaseTransform
from typing import Union
from torch_geometric.loader import LinkNeighborLoader
from sklearn.metrics import f1_score
import json
from torch_geometric.nn import to_hetero, summary
from torch_geometric.utils import degree
import wandb
import sys
import time
# import argparse
import random

## Formatting Data

In [10]:
inPath = data_path + '/small/HI-Small_Trans.csv'
outPath = data_path + "/small/formatted_transactions.csv"
if not os.path.exists(outPath) :
  raw = pd.read_csv(inPath)
  raw['From Bank'] = raw['From Bank'].astype(str)
  raw['To Bank'] = raw['To Bank'].astype(str)
  formatted = pd.DataFrame(columns = ['EdgeID', 'from_id', 'to_id', 'Timestamp', 'Amount Sent', 'Sent Currency', 'Amount Received', 'Received Currency', 'Payment Format', 'Is Laundering'])

  formatted['EdgeID'] = raw.index

  formatted['from_id'] = raw['From Bank'] + raw['Account']
  formatted['to_id'] = raw['To Bank'] + raw['Account.1']
  unique_accounts = pd.concat([formatted['from_id'], formatted['to_id']]).unique()
  account_label_encoder = LabelEncoder()
  account_label_encoder.fit(unique_accounts)
  formatted['from_id'] = account_label_encoder.transform(formatted['from_id'])
  formatted['to_id'] = account_label_encoder.transform(formatted['to_id'])

  raw['Timestamp'] = pd.to_datetime(raw['Timestamp'])
  min_time = raw['Timestamp'].min()
  raw['Timestamp'] = (raw['Timestamp'] - min_time)
  formatted['Timestamp'] = raw['Timestamp'].dt.total_seconds()


  formatted['Amount Sent'] = raw['Amount Paid']

  formatted['Sent Currency'] = raw['Payment Currency']

  formatted['Amount Received'] = raw['Amount Received']

  formatted['Received Currency'] = raw['Receiving Currency']

  formatted['Payment Format'] = raw['Payment Format']

  formatted['Is Laundering'] = raw['Is Laundering']

  categorical_columns = ['Sent Currency', 'Received Currency', 'Payment Format']
  label_encoders = {}
  label_encoders['account'] = account_label_encoder
  for column in categorical_columns:
      label_encoders[column] = LabelEncoder()
      formatted[column] = label_encoders[column].fit_transform(formatted[column])
      label_encoders[column] = LabelEncoder()

  formatted.to_csv(outPath, index=False)

# df = pd.read_csv(inPath)
# f_df = pd.read_csv(outPath)

In [11]:
# df.head()

In [12]:
# f_df = pd.read_csv(outPath)
# f_df.head()

In [13]:
# f_df.iloc[1]

# Data Preprocessing

## Data Utility Funcs

In [14]:
def to_adj_nodes_with_times(data):
    num_nodes = data.num_nodes
    timestamps = torch.zeros((data.edge_index.shape[1], 1)) if data.timestamps is None else data.timestamps.reshape((-1,1))
    edges = torch.cat((data.edge_index.T, timestamps), dim=1) if not isinstance(data, HeteroData) else torch.cat((data['node', 'to', 'node'].edge_index.T, timestamps), dim=1)
    adj_list_out = dict([(i, []) for i in range(num_nodes)])
    adj_list_in = dict([(i, []) for i in range(num_nodes)])
    for u,v,t in edges:
        u,v,t = int(u), int(v), int(t)
        adj_list_out[u] += [(v, t)]
        adj_list_in[v] += [(u, t)]
    return adj_list_in, adj_list_out




In [15]:
def to_adj_edges_with_times(data):
    num_nodes = data.num_nodes
    timestamps = torch.zeros((data.edge_index.shape[1], 1)) if data.timestamps is None else data.timestamps.reshape((-1,1))
    edges = torch.cat((data.edge_index.T, timestamps), dim=1)
    # calculate adjacent edges with times per node
    adj_edges_out = dict([(i, []) for i in range(num_nodes)])
    adj_edges_in = dict([(i, []) for i in range(num_nodes)])
    for i, (u,v,t) in enumerate(edges):
        u,v,t = int(u), int(v), int(t)
        adj_edges_out[u] += [(i, v, t)]
        adj_edges_in[v] += [(i, u, t)]
    return adj_edges_in, adj_edges_out


In [16]:
def ports(edge_index, adj_list):
    ports = torch.zeros(edge_index.shape[1], 1)
    ports_dict = {}
    for v, nbs in adj_list.items():
        if len(nbs) < 1: continue
        a = np.array(nbs)
        a = a[a[:, -1].argsort()]
        _, idx = np.unique(a[:,[0]],return_index=True,axis=0)
        nbs_unique = a[np.sort(idx)][:,0]
        for i, u in enumerate(nbs_unique):
            ports_dict[(u,v)] = i
    for i, e in enumerate(edge_index.T):
        ports[i] = ports_dict[tuple(e.numpy())]
    return ports



In [17]:
def time_deltas(data, adj_edges_list):
    time_deltas = torch.zeros(data.edge_index.shape[1], 1)
    if data.timestamps is None:
        return time_deltas
    for v, edges in adj_edges_list.items():
        if len(edges) < 1: continue
        a = np.array(edges)
        a = a[a[:, -1].argsort()]
        a_tds = [0] + [a[i+1,-1] - a[i,-1] for i in range(a.shape[0]-1)]
        tds = np.hstack((a[:,0].reshape(-1,1), np.array(a_tds).reshape(-1,1)))
        for i,td in tds:
            time_deltas[i] = td
    return time_deltas


In [18]:
def z_norm(data):
    std = data.std(0).unsqueeze(0)
    std = torch.where(std == 0, torch.tensor(1, dtype=torch.float32).cpu(), std)
    return (data - data.mean(0).unsqueeze(0)) / std

In [19]:
def create_hetero_obj(x,  y,  edge_index,  edge_attr, timestamps, ports):
    '''Creates a heterogenous graph object for reverse message passing'''
    data = HeteroGraphData()

    data['node'].x = x
    data['node', 'to', 'node'].edge_index = edge_index
    data['node', 'rev_to', 'node'].edge_index = edge_index.flipud()
    data['node', 'to', 'node'].edge_attr = edge_attr
    data['node', 'rev_to', 'node'].edge_attr = edge_attr
    if ports:
        #swap the in- and outgoing port numberings for the reverse edges
        data['node', 'rev_to', 'node'].edge_attr[:, [-1, -2]] = data['node', 'rev_to', 'node'].edge_attr[:, [-2, -1]]
    data['node', 'to', 'node'].y = y
    data['node', 'to', 'node'].timestamps = timestamps

    return data

## GrapgData Class
for homogeneous graphs

In [20]:
class GraphData(Data):
    '''This is the homogenous graph object we use for GNN training if reverse MP is not enabled'''
    def __init__(
        self, x: OptTensor = None, edge_index: OptTensor = None, edge_attr: OptTensor = None, y: OptTensor = None, pos: OptTensor = None,
        readout: str = 'edge',
        num_nodes: int = None,
        timestamps: OptTensor = None,
        node_timestamps: OptTensor = None,
        **kwargs
        ):
        super().__init__(x, edge_index, edge_attr, y, pos, **kwargs)
        self.readout = readout
        self.loss_fn = 'ce'
        self.num_nodes = int(self.x.shape[0])
        self.node_timestamps = node_timestamps
        if timestamps is not None:
            self.timestamps = timestamps
        elif edge_attr is not None:
            self.timestamps = edge_attr[:,0].clone()
        else:
            self.timestamps = None

    def add_ports(self):
        '''Adds port numberings to the edge features'''
        reverse_ports = True
        adj_list_in, adj_list_out = to_adj_nodes_with_times(self)
        in_ports = ports(self.edge_index, adj_list_in)
        out_ports = [ports(self.edge_index.flipud(), adj_list_out)] if reverse_ports else []
        self.edge_attr = torch.cat([self.edge_attr, in_ports] + out_ports, dim=1)
        return self

    def add_time_deltas(self):
        '''Adds time deltas (i.e. the time between subsequent transactions) to the edge features'''
        reverse_tds = True
        adj_list_in, adj_list_out = to_adj_edges_with_times(self)
        in_tds = time_deltas(self, adj_list_in)
        out_tds = [time_deltas(self, adj_list_out)] if reverse_tds else []
        self.edge_attr = torch.cat([self.edge_attr, in_tds] + out_tds, dim=1)
        return self






## HeteroGraphData Class
for heterogeneous graphs when reverse message pasing is enabled.

In [21]:
class HeteroGraphData(HeteroData):
    '''This is the heterogenous graph object we use for GNN training if reverse MP is enabled'''
    def __init__(
        self,
        readout: str = 'edge',
        **kwargs
        ):
        super().__init__(**kwargs)
        self.readout = readout

    @property
    def num_nodes(self):
        return self['node'].x.shape[0]

    @property
    def timestamps(self):
        return self['node', 'to', 'node'].timestamps

    def add_ports(self):
        '''Adds port numberings to the edge features'''
        adj_list_in, adj_list_out = to_adj_nodes_with_times(self)
        in_ports = ports(self['node', 'to', 'node'].edge_index, adj_list_in)
        out_ports = ports(self['node', 'rev_to', 'node'].edge_index, adj_list_out)
        self['node', 'to', 'node'].edge_attr = torch.cat([self['node', 'to', 'node'].edge_attr, in_ports], dim=1)
        self['node', 'rev_to', 'node'].edge_attr = torch.cat([self['node', 'rev_to', 'node'].edge_attr, out_ports], dim=1)
        return self

    def add_time_deltas(self):
        '''Adds time deltas (i.e. the time between subsequent transactions) to the edge features'''
        adj_list_in, adj_list_out = to_adj_edges_with_times(self)
        in_tds = time_deltas(self, adj_list_in)
        out_tds = time_deltas(self, adj_list_out)
        self['node', 'to', 'node'].edge_attr = torch.cat([self['node', 'to', 'node'].edge_attr, in_tds], dim=1)
        self['node', 'rev_to', 'node'].edge_attr = torch.cat([self['node', 'rev_to', 'node'].edge_attr, out_tds], dim=1)
        return self


## Data handler

In [22]:
# import pandas as pd
# import numpy as np
# import torch
# import itertools
# from data_util import GraphData, HeteroData, z_norm, create_hetero_obj

def get_data(model : 'str',
             formatted_data_path,
             ports = False,
             tds = False,
             reverse_mp = False,
             ):
    '''Loads the AML transaction data.

    1. The data is loaded from the csv and the necessary features are chosen.
    2. The data is split into training, validation and test data.
    3. PyG Data objects are created with the respective data splits.
    '''

    transaction_file = formatted_data_path
    df_edges = pd.read_csv(transaction_file)

    print(f'Available Edge Features: {df_edges.columns.tolist()}')

    df_edges['Timestamp'] = df_edges['Timestamp'] - df_edges['Timestamp'].min()

    max_n_id = df_edges.loc[:, ['from_id', 'to_id']].to_numpy().max() + 1 # no. of unique accounts
    df_nodes = pd.DataFrame({'NodeID': np.arange(max_n_id), 'Feature': np.ones(max_n_id)}) # node feature dataframe for each account
    timestamps = torch.Tensor(df_edges['Timestamp'].to_numpy())
    y = torch.LongTensor(df_edges['Is Laundering'].to_numpy())

    print(f"Illicit ratio = {sum(y)} / {len(y)} = {sum(y) / len(y) * 100:.2f}%")
    print(f"Number of nodes (holdings doing transcations) = {df_nodes.shape[0]}")
    print(f"Number of transactions = {df_edges.shape[0]}")

    edge_features = ['Timestamp', 'Amount Received', 'Received Currency', 'Payment Format']
    node_features = ['Feature']

    print(f'Edge features being used: {edge_features}')
    print(f'Node features being used: {node_features} ("Feature" is a placeholder feature of all 1s)')

    x = torch.tensor(df_nodes.loc[:, node_features].to_numpy()).float()
    edge_index = torch.LongTensor(df_edges.loc[:, ['from_id', 'to_id']].to_numpy().T)
    edge_attr = torch.tensor(df_edges.loc[:, edge_features].to_numpy()).float()

    n_days = int(timestamps.max() / (3600 * 24) + 1)
    n_samples = y.shape[0]
    print(f'number of days and transactions in the data: {n_days} days, {n_samples} transactions')

    #data splitting
    daily_irs, weighted_daily_irs, daily_inds, daily_trans = [], [], [], [] #irs = illicit ratios, inds = indices, trans = transactions
    for day in range(n_days):
        l = day * 24 * 3600
        r = (day + 1) * 24 * 3600
        day_inds = torch.where((timestamps >= l) & (timestamps < r))[0]
        daily_irs.append(y[day_inds].float().mean())
        weighted_daily_irs.append(y[day_inds].float().mean() * day_inds.shape[0] / n_samples)
        daily_inds.append(day_inds)
        daily_trans.append(day_inds.shape[0])

    split_per = [0.6, 0.2, 0.2]
    daily_totals = np.array(daily_trans)
    d_ts = daily_totals
    I = list(range(len(d_ts)))
    split_scores = dict()
    for i,j in itertools.combinations(I, 2):
        if j >= i:
            split_totals = [d_ts[:i].sum(), d_ts[i:j].sum(), d_ts[j:].sum()]
            split_totals_sum = np.sum(split_totals)
            split_props = [v/split_totals_sum for v in split_totals]
            split_error = [abs(v-t)/t for v,t in zip(split_props, split_per)]
            score = max(split_error) #- (split_totals_sum/total) + 1
            split_scores[(i,j)] = score
        else:
            continue

    i,j = min(split_scores, key=split_scores.get)
    #split contains a list for each split (train, validation and test) and each list contains the days that are part of the respective split
    split = [list(range(i)), list(range(i, j)), list(range(j, len(daily_totals)))]
    print(f'Calculate split: {split}')

    #Now, we seperate the transactions based on their indices in the timestamp array
    split_inds = {k: [] for k in range(3)}
    for i in range(3):
        for day in split[i]:
            split_inds[i].append(daily_inds[day]) #split_inds contains a list for each split (tr,val,te) which contains the indices of each day seperately

    tr_inds = torch.cat(split_inds[0])
    val_inds = torch.cat(split_inds[1])
    te_inds = torch.cat(split_inds[2])

    print(f"Total train samples: {tr_inds.shape[0] / y.shape[0] * 100 :.2f}% || IR: "
            f"{y[tr_inds].float().mean() * 100 :.2f}% || Train days: {split[0][:5]}")
    print(f"Total val samples: {val_inds.shape[0] / y.shape[0] * 100 :.2f}% || IR: "
        f"{y[val_inds].float().mean() * 100:.2f}% || Val days: {split[1][:5]}")
    print(f"Total test samples: {te_inds.shape[0] / y.shape[0] * 100 :.2f}% || IR: "
        f"{y[te_inds].float().mean() * 100:.2f}% || Test days: {split[2][:5]}")

    #Creating the final data objects
    tr_x, val_x, te_x = x, x, x
    e_tr = tr_inds.numpy()
    e_val = np.concatenate([tr_inds, val_inds])

    tr_edge_index,  tr_edge_attr,  tr_y,  tr_edge_times  = edge_index[:,e_tr],  edge_attr[e_tr],  y[e_tr],  timestamps[e_tr]
    val_edge_index, val_edge_attr, val_y, val_edge_times = edge_index[:,e_val], edge_attr[e_val], y[e_val], timestamps[e_val]
    te_edge_index,  te_edge_attr,  te_y,  te_edge_times  = edge_index,          edge_attr,        y,        timestamps

    tr_data = GraphData (x=tr_x,  y=tr_y,  edge_index=tr_edge_index,  edge_attr=tr_edge_attr,  timestamps=tr_edge_times )
    val_data = GraphData(x=val_x, y=val_y, edge_index=val_edge_index, edge_attr=val_edge_attr, timestamps=val_edge_times)
    te_data = GraphData (x=te_x,  y=te_y,  edge_index=te_edge_index,  edge_attr=te_edge_attr,  timestamps=te_edge_times )

    #Adding ports and time-deltas if applicable
    if ports:
        print(f"Start: adding ports")
        tr_data.add_ports()
        val_data.add_ports()
        te_data.add_ports()
        print(f"Done: adding ports")
    if tds:
        print(f"Start: adding time-deltas")
        tr_data.add_time_deltas()
        val_data.add_time_deltas()
        te_data.add_time_deltas()
        print(f"Done: adding time-deltas")

    #Normalize data
    tr_data.x = val_data.x = te_data.x = z_norm(tr_data.x)
    if not model == 'rgcn':
        tr_data.edge_attr, val_data.edge_attr, te_data.edge_attr = z_norm(tr_data.edge_attr), z_norm(val_data.edge_attr), z_norm(te_data.edge_attr)
    else:
        tr_data.edge_attr[:, :-1], val_data.edge_attr[:, :-1], te_data.edge_attr[:, :-1] = z_norm(tr_data.edge_attr[:, :-1]), z_norm(val_data.edge_attr[:, :-1]), z_norm(te_data.edge_attr[:, :-1])

    #Create heterogenous if reverese MP is enabled
    #TODO: if I observe wierd behaviour, maybe add .detach.clone() to all torch tensors, but I don't think they're attached to any computation graph just yet
    if reverse_mp:
        tr_data = create_hetero_obj(tr_data.x,  tr_data.y,  tr_data.edge_index,  tr_data.edge_attr, tr_data.timestamps, ports)
        val_data = create_hetero_obj(val_data.x,  val_data.y,  val_data.edge_index,  val_data.edge_attr, val_data.timestamps, ports)
        te_data = create_hetero_obj(te_data.x,  te_data.y,  te_data.edge_index,  te_data.edge_attr, te_data.timestamps, ports)

    print(f'train data object: {tr_data}')
    print(f'validation data object: {val_data}')
    print(f'test data object: {te_data}')

    return tr_data, val_data, te_data, tr_inds, val_inds, te_inds


In [23]:
# get_data(
#     model = 'gat',
#     formatted_data_path = outPath,
#     ports = True,
#     tds = True,
#     reverse_mp = False
# )

### Test run for data processing

Available Edge Features: ['EdgeID', 'from_id', 'to_id', 'Timestamp', 'Amount Sent', 'Sent Currency', 'Amount Received', 'Received Currency', 'Payment Format', 'Is Laundering'] \
Illicit ratio = 5177 / 5078345 = 0.10% \
Number of nodes (holdings doing transcations) = 515088 \
Number of transactions = 5078345 \
Edge features being used: ['Timestamp', 'Amount Received', 'Received Currency', 'Payment Format'] \
Node features being used: ['Feature'] ("Feature" is a placeholder feature of all 1s) \
number of days and transactions in the data: 18 days, 5078345 transactions \
Calculate split: [[0, 1, 2, 3, 4, 5], [6, 7], [8, 9, 10, 11, 12, 13, 14, 15, 16, 17]] \
Total train samples: 63.98% || IR: 0.08% || Train days: [0, 1, 2, 3, 4] \
Total val samples: 19.01% || IR: 0.11% || Val days: [6, 7] \
Total test samples: 17.01% || IR: 0.19% || Test days: [8, 9, 10, 11, 12] \
Start: adding ports \
Done: adding ports \
Start: adding time-deltas \
Done: adding time-deltas \
train data object: GraphData(x=[515088, 1], edge_index=[2, 3248921], edge_attr=[3248921, 8], y=[3248921], readout='edge', loss_fn='ce', num_nodes=515088, timestamps=[3248921]) \
validation data object: GraphData(x=[515088, 1], edge_index=[2, 4214445], edge_attr=[4214445, 8], y=[4214445], readout='edge', loss_fn='ce', num_nodes=515088, timestamps=[4214445]) \
test data object: GraphData(x=[515088, 1], edge_index=[2, 5078345], edge_attr=[5078345, 8], y=[5078345], readout='edge', loss_fn='ce', num_nodes=515088, timestamps=[5078345]) \
(GraphData(x=[515088, 1], edge_index=[2, 3248921], edge_attr=[3248921, 8], y=[3248921], readout='edge', loss_fn='ce', num_nodes=515088, timestamps=[3248921]), \
 GraphData(x=[515088, 1], edge_index=[2, 4214445], edge_attr=[4214445, 8], y=[4214445], readout='edge', loss_fn='ce', num_nodes=515088, timestamps=[4214445]), \
 GraphData(x=[515088, 1], edge_index=[2, 5078345], edge_attr=[5078345, 8], y=[5078345], readout='edge', loss_fn='ce', num_nodes=515088, timestamps=[5078345]), \
 tensor([      0,       1,       2,  ..., 3249981, 3249982, 3249983]), \
 tensor([ 695463,  695464,  695465,  ..., 4215347, 4215348, 4215349]), \
 tensor([ 812708,  812709,  812710,  ..., 5033972, 5033973, 5033974])) \

# Architectures

In [24]:
# import torch.nn as nn
# from torch_geometric.nn import GINEConv, BatchNorm, Linear, GATConv, PNAConv, RGCNConv
# import torch.nn.functional as F
# import torch
# import logging

## GIN

In [25]:
class GINe(torch.nn.Module):
    def __init__(self, num_features, num_gnn_layers, n_classes=2,
                n_hidden=100, edge_updates=False, residual=True,
                edge_dim=None, dropout=0.0, final_dropout=0.5):
        super().__init__()
        self.n_hidden = n_hidden
        self.num_gnn_layers = num_gnn_layers
        self.edge_updates = edge_updates
        self.final_dropout = final_dropout

        self.node_emb = nn.Linear(num_features, n_hidden)
        self.edge_emb = nn.Linear(edge_dim, n_hidden)

        self.convs = nn.ModuleList()
        self.emlps = nn.ModuleList()
        self.batch_norms = nn.ModuleList()
        for _ in range(self.num_gnn_layers):
            conv = GINEConv(nn.Sequential(
                nn.Linear(self.n_hidden, self.n_hidden),
                nn.ReLU(),
                nn.Linear(self.n_hidden, self.n_hidden)
                ), edge_dim=self.n_hidden)
            if self.edge_updates: self.emlps.append(nn.Sequential(
                nn.Linear(3 * self.n_hidden, self.n_hidden),
                nn.ReLU(),
                nn.Linear(self.n_hidden, self.n_hidden),
            ))
            self.convs.append(conv)
            self.batch_norms.append(BatchNorm(n_hidden))

        self.mlp = nn.Sequential(Linear(n_hidden*3, 50), nn.ReLU(), nn.Dropout(self.final_dropout),Linear(50, 25), nn.ReLU(), nn.Dropout(self.final_dropout),
                              Linear(25, n_classes))

    def forward(self, x, edge_index, edge_attr):
        src, dst = edge_index

        x = self.node_emb(x)
        edge_attr = self.edge_emb(edge_attr)

        for i in range(self.num_gnn_layers):
            x = (x + F.relu(self.batch_norms[i](self.convs[i](x, edge_index, edge_attr)))) / 2
            if self.edge_updates:
                edge_attr = edge_attr + self.emlps[i](torch.cat([x[src], x[dst], edge_attr], dim=-1)) / 2

        x = x[edge_index.T].reshape(-1, 2 * self.n_hidden).relu()
        x = torch.cat((x, edge_attr.view(-1, edge_attr.shape[1])), 1)
        out = x

        return self.mlp(out)



## GAT

In [26]:
class GATe(torch.nn.Module):
    def __init__(self, num_features, num_gnn_layers, n_classes=2, n_hidden=100, n_heads=4, edge_updates=False, edge_dim=None, dropout=0.0, final_dropout=0.5):
        super().__init__()
        # GAT specific code
        tmp_out = n_hidden // n_heads
        n_hidden = tmp_out * n_heads

        self.n_hidden = n_hidden
        self.n_heads = n_heads
        self.num_gnn_layers = num_gnn_layers
        self.edge_updates = edge_updates
        self.dropout = dropout
        self.final_dropout = final_dropout

        self.node_emb = nn.Linear(num_features, n_hidden)
        self.edge_emb = nn.Linear(edge_dim, n_hidden)

        self.convs = nn.ModuleList()
        self.emlps = nn.ModuleList()
        self.batch_norms = nn.ModuleList()

        for _ in range(self.num_gnn_layers):
            conv = GATConv(self.n_hidden, tmp_out, self.n_heads, concat = True, dropout = self.dropout, add_self_loops = True, edge_dim=self.n_hidden)
            if self.edge_updates: self.emlps.append(nn.Sequential(nn.Linear(3 * self.n_hidden, self.n_hidden),nn.ReLU(),nn.Linear(self.n_hidden, self.n_hidden),))
            self.convs.append(conv)
            self.batch_norms.append(BatchNorm(n_hidden))

        self.mlp = nn.Sequential(Linear(n_hidden*3, 50), nn.ReLU(), nn.Dropout(self.final_dropout),Linear(50, 25), nn.ReLU(), nn.Dropout(self.final_dropout),Linear(25, n_classes))

    def forward(self, x, edge_index, edge_attr):
        src, dst = edge_index

        x = self.node_emb(x)
        edge_attr = self.edge_emb(edge_attr)

        for i in range(self.num_gnn_layers):
            x = (x + F.relu(self.batch_norms[i](self.convs[i](x, edge_index, edge_attr)))) / 2
            if self.edge_updates:
                edge_attr = edge_attr + self.emlps[i](torch.cat([x[src], x[dst], edge_attr], dim=-1)) / 2

        # logging.debug(f"x.shape = {x.shape}, x[edge_index.T].shape = {x[edge_index.T].shape}")
        x = x[edge_index.T].reshape(-1, 2 * self.n_hidden).relu()
        # logging.debug(f"x.shape = {x.shape}")
        x = torch.cat((x, edge_attr.view(-1, edge_attr.shape[1])), 1)
        # logging.debug(f"x.shape = {x.shape}")
        out = x

        return self.mlp(out)



## PNA

In [27]:
class PNA(torch.nn.Module):
    def __init__(self, num_features, num_gnn_layers, n_classes=2,
                n_hidden=100, edge_updates=True,
                edge_dim=None, dropout=0.0, final_dropout=0.5, deg=None):
        super().__init__()
        n_hidden = int((n_hidden // 5) * 5)
        self.n_hidden = n_hidden
        self.num_gnn_layers = num_gnn_layers
        self.edge_updates = edge_updates
        self.final_dropout = final_dropout

        aggregators = ['mean', 'min', 'max', 'std']
        scalers = ['identity', 'amplification', 'attenuation']

        self.node_emb = nn.Linear(num_features, n_hidden)
        self.edge_emb = nn.Linear(edge_dim, n_hidden)

        self.convs = nn.ModuleList()
        self.emlps = nn.ModuleList()
        self.batch_norms = nn.ModuleList()
        for _ in range(self.num_gnn_layers):
            conv = PNAConv(in_channels=n_hidden, out_channels=n_hidden,
                           aggregators=aggregators, scalers=scalers, deg=deg,
                           edge_dim=n_hidden, towers=5, pre_layers=1, post_layers=1,
                           divide_input=False)
            if self.edge_updates: self.emlps.append(nn.Sequential(
                nn.Linear(3 * self.n_hidden, self.n_hidden),
                nn.ReLU(),
                nn.Linear(self.n_hidden, self.n_hidden),
            ))
            self.convs.append(conv)
            self.batch_norms.append(BatchNorm(n_hidden))

        self.mlp = nn.Sequential(Linear(n_hidden*3, 50), nn.ReLU(), nn.Dropout(self.final_dropout),Linear(50, 25), nn.ReLU(), nn.Dropout(self.final_dropout),
                              Linear(25, n_classes))

    def forward(self, x, edge_index, edge_attr):
        src, dst = edge_index

        x = self.node_emb(x)
        edge_attr = self.edge_emb(edge_attr)

        for i in range(self.num_gnn_layers):
            x = (x + F.relu(self.batch_norms[i](self.convs[i](x, edge_index, edge_attr)))) / 2
            if self.edge_updates:
                edge_attr = edge_attr + self.emlps[i](torch.cat([x[src], x[dst], edge_attr], dim=-1)) / 2

        logging.debug(f"x.shape = {x.shape}, x[edge_index.T].shape = {x[edge_index.T].shape}")
        x = x[edge_index.T].reshape(-1, 2 * self.n_hidden).relu()
        logging.debug(f"x.shape = {x.shape}")
        x = torch.cat((x, edge_attr.view(-1, edge_attr.shape[1])), 1)
        logging.debug(f"x.shape = {x.shape}")
        out = x
        return self.mlp(out)



## RGCN

In [28]:
class RGCN(nn.Module):
    def __init__(self, num_features, edge_dim, num_relations, num_gnn_layers, n_classes=2,
                n_hidden=100, edge_update=False,
                residual=True,
                dropout=0.0, final_dropout=0.5, n_bases=-1):
        super(RGCN, self).__init__()

        self.num_features = num_features
        self.num_gnn_layers = num_gnn_layers
        self.n_hidden = n_hidden
        self.residual = residual
        self.dropout = dropout
        self.final_dropout = final_dropout
        self.n_classes = n_classes
        self.edge_update = edge_update
        self.num_relations = num_relations
        self.n_bases = n_bases

        self.node_emb = nn.Linear(num_features, n_hidden)
        self.edge_emb = nn.Linear(edge_dim, n_hidden)

        self.convs = nn.ModuleList()
        self.bns = nn.ModuleList()
        self.mlp = nn.ModuleList()

        if self.edge_update:
            self.emlps = nn.ModuleList()
            self.emlps.append(nn.Sequential(
                nn.Linear(3 * self.n_hidden, self.n_hidden),
                nn.ReLU(),
                nn.Linear(self.n_hidden, self.n_hidden),
            ))

        for _ in range(self.num_gnn_layers):
            conv = RGCNConv(self.n_hidden, self.n_hidden, num_relations, num_bases=self.n_bases)
            self.convs.append(conv)
            self.bns.append(nn.BatchNorm1d(self.n_hidden))

            if self.edge_update:
                self.emlps.append(nn.Sequential(
                    nn.Linear(3 * self.n_hidden, self.n_hidden),
                    nn.ReLU(),
                    nn.Linear(self.n_hidden, self.n_hidden),
                ))

        self.mlp = nn.Sequential(Linear(n_hidden*3, 50), nn.ReLU(), nn.Dropout(self.final_dropout), Linear(50, 25), nn.ReLU(), nn.Dropout(self.final_dropout),
                              Linear(25, n_classes))

    def reset_parameters(self):
        for m in self.modules():
            if isinstance(m, nn.Linear):
                m.reset_parameters()
            elif isinstance(m, RGCNConv):
                m.reset_parameters()
            elif isinstance(m, nn.BatchNorm1d):
                m.reset_parameters()

    def forward(self, x, edge_index, edge_attr):
        edge_type = edge_attr[:, -1].long()
        #edge_attr = edge_attr[:, :-1]
        src, dst = edge_index

        x = self.node_emb(x)
        edge_attr = self.edge_emb(edge_attr)

        for i in range(self.num_gnn_layers):
            x =  (x + F.relu(self.bns[i](self.convs[i](x, edge_index, edge_type)))) / 2
            if self.edge_update:
                edge_attr = (edge_attr + F.relu(self.emlps[i](torch.cat([x[src], x[dst], edge_attr], dim=-1)))) / 2

        x = x[edge_index.T].reshape(-1, 2 * self.n_hidden).relu()
        x = torch.cat((x, edge_attr.view(-1, edge_attr.shape[1])), 1)
        x = self.mlp(x)
        out = x

        return x

# Training Functions

## Model Settings

In [29]:
model_settings = {
      "gin": {
        "params": {
          "lr": 0.006213266113989207, "n_hidden": 66.00315515631006, "n_mlp_layers": 1, "n_gnn_layers": 2, "loss": "ce",
          "w_ce1": 1.0000182882773443, "w_ce2": 6.275014431494497, "norm_method": "z_normalize", "dropout": 0.00983468338330501,
          "final_dropout": 0.10527690625126304
        },
        "bayes_opt_params": {
          "lr": [0.002, 0.007], "n_hidden": [66.0, 66.01], "n_mlp_layers": [1, 1.001], "n_gnn_layers": [2.0, 2.001],
          "loss": [0.0, 0.1], "w_ce1": [1, 1.001], "w_ce2": [6, 12], "norm_method": [0, 0.001], "dropout": [0, 0.05],
          "final_dropout": [0, 0.2]
        },
        "header": "run,tb,lr,n_hidden,n_mlp_layers,n_gnn_layers,loss,w_ce1,w_ce2,norm_method,dropout,final_dropout,epoch,tr_acc,tr_prec,tr_rec,tr_f1,tr_auc,val_acc,val_prec,val_rec,val_f1,val_auc,te_acc,te_prec,te_rec,te_f1,te_auc\n"
      },

      "pna": {
        "params": {
          "lr": 0.0006116418195373612, "n_hidden": 20, "n_mlp_layers": 1, "n_gnn_layers": 2, "loss": "ce", "w_ce1": 1.0003967674742307,
          "w_ce2": 7.077633468006714, "norm_method": "z_normalize", "dropout": 0.08340440094051481, "final_dropout": 0.28812979737686323
        },
        "bayes_opt_params": {
          "lr": [0.0001, 0.001], "n_hidden": [16, 64], "n_mlp_layers": [1, 1.001], "n_gnn_layers": [2.00, 2.01], "loss": [0.0, 0.1],
          "w_ce1": [1, 1.001], "w_ce2": [6, 12], "norm_method": [0, 0.1], "dropout": [0.0, 0.2], "final_dropout": [0.0, 0.4]
        },
        "header": "run,tb,lr,n_hidden,n_mlp_layers,n_gnn_layers,loss,w_ce1,w_ce2,norm_method,dropout,final_dropout,epoch,tr_acc,tr_prec,tr_rec,tr_f1,tr_auc,val_acc,val_prec,val_rec,val_f1,val_auc,te_acc,te_prec,te_rec,te_f1,te_auc\n"
      },

      "gat": {
        "params": {
          "lr": 0.006, "n_hidden": 64, "n_heads": 4, "n_mlp_layers": 1, "n_gnn_layers": 2, "loss": "ce", "w_ce1": 1, "w_ce2": 6,
          "norm_method": "z_normalize", "dropout": 0.009, "final_dropout": 0.1
        },
        "bayes_opt_params": {
          "lr": [0.01, 0.04], "n_hidden": [4, 24], "n_heads": [1.5, 4.5], "n_mlp_layers": [1, 1.001], "n_gnn_layers": [3, 7],
          "loss": [0, 0.1], "w_ce1": [1, 1.001], "w_ce2": [1, 10], "norm_method": [0, 0.1], "dropout": [0, 0.5],
          "final_dropout": [0, 0.8]
        },
        "header": "run,tb,lr,n_hidden,n_heads,n_mlp_layers,n_gnn_layers,loss,w_ce1,w_ce2,norm_method,dropout,final_dropout,epoch,tr_acc,tr_prec,tr_rec,tr_f1,tr_auc,val_acc,val_prec,val_rec,val_f1,val_auc,te_acc,te_prec,te_rec,te_f1,te_auc\n"
      },

      "mlp": {
        "params": {
          "lr": 0.006213266113989207, "n_hidden": 66.00315515631006, "n_mlp_layers": 1, "n_gnn_layers": 2, "loss": "ce", "w_ce1": 1.0000182882773443,
          "w_ce2": 9.23, "norm_method": "z_normalize", "dropout": 0.00983468338330501, "final_dropout": 0.10527690625126304
        },
        "bayes_opt_params": {
          "lr": [0.006, 0.0064], "n_hidden": [66.0, 66.01], "n_mlp_layers": [1, 1.001], "n_gnn_layers": [2.0, 2.001], "loss": [0.0, 0.1],
          "w_ce1": [1, 1.001], "w_ce2": [6, 12], "norm_method": [0, 0.001], "dropout": [0, 0.05], "final_dropout": [0, 0.2]
        },
        "header": "run,tb,lr,n_hidden,n_mlp_layers,n_gnn_layers,loss,w_ce1,w_ce2,norm_method,dropout,final_dropout,epoch,tr_acc,tr_prec,tr_rec,tr_f1,tr_auc,val_acc,val_prec,val_rec,val_f1,val_auc,te_acc,te_prec,te_rec,te_f1,te_auc\n"
      },
      "rgcn": {
        "params": {
          "lr": 0.006213266113989207, "n_hidden": 66.00315515631006, "n_mlp_layers": 1, "n_gnn_layers": 2, "loss": "ce", "w_ce1": 1.0000182882773443,
          "w_ce2": 9.23, "norm_method": "z_normalize", "dropout": 0.00983468338330501, "final_dropout": 0.10527690625126304
        },
        "bayes_opt_params": {
          "lr": [0.006, 0.0064], "n_hidden": [66.0, 66.01], "n_mlp_layers": [1, 1.001], "n_gnn_layers": [2.0, 2.001], "loss": [0.0, 0.1],
          "w_ce1": [1, 1.001], "w_ce2": [6, 12], "norm_method": [0, 0.001], "dropout": [0, 0.05], "final_dropout": [0, 0.2]
        },
        "header": "run,tb,lr,n_hidden,n_mlp_layers,n_gnn_layers,loss,w_ce1,w_ce2,norm_method,dropout,final_dropout,epoch,tr_acc,tr_prec,tr_rec,tr_f1,tr_auc,val_acc,val_prec,val_rec,val_f1,val_auc,te_acc,te_prec,te_rec,te_f1,te_auc\n"
      }
    }

## Training utils

In [30]:
# import torch
# import tqdm
# from torch_geometric.transforms import BaseTransform
# from typing import Union
# from torch_geometric.data import Data, HeteroData
# from torch_geometric.loader import LinkNeighborLoader
# from sklearn.metrics import f1_score
# import json

In [31]:
class AddEgoIds(BaseTransform):
    r"""Add IDs to the centre nodes of the batch.
    """
    def __init__(self):
        pass

    def __call__(self, data: Union[Data, HeteroData]):
        x = data.x if not isinstance(data, HeteroData) else data['node'].x
        device = x.device
        ids = torch.zeros((x.shape[0], 1), device=device)
        if not isinstance(data, HeteroData):
            nodes = torch.unique(data.edge_label_index.view(-1)).to(device)
        else:
            nodes = torch.unique(data['node', 'to', 'node'].edge_label_index.view(-1)).to(device)
        ids[nodes] = 1
        if not isinstance(data, HeteroData):
            data.x = torch.cat([x, ids], dim=1)
        else:
            data['node'].x = torch.cat([x, ids], dim=1)

        return data


In [32]:
def extract_param(parameter_name: str, model : str) -> float:
    """
    Extract the value of the specified parameter for the given model.

    Args:
    - parameter_name (str): Name of the parameter (e.g., "lr").
    - args (argparser): Arguments given to this specific run.

    Returns:
    - float: Value of the specified parameter.
    """
    # file_path = './model_settings.json'
    # with open(file_path, "r") as file:
    #     data = json.load(file)
    # return data.get(args.model, {}).get("params", {}).get(parameter_name, None)
    return model_settings[model]["params"][parameter_name]


In [33]:

def add_arange_ids(data_list):
    '''
    Add the index as an id to the edge features to find seed edges in training, validation and testing.

    Args:
    - data_list (str): List of tr_data, val_data and te_data.
    '''
    for data in data_list:
        if isinstance(data, HeteroData):
            data['node', 'to', 'node'].edge_attr = torch.cat([torch.arange(data['node', 'to', 'node'].edge_attr.shape[0]).view(-1, 1), data['node', 'to', 'node'].edge_attr], dim=1)
            offset = data['node', 'to', 'node'].edge_attr.shape[0]
            data['node', 'rev_to', 'node'].edge_attr = torch.cat([torch.arange(offset, data['node', 'rev_to', 'node'].edge_attr.shape[0] + offset).view(-1, 1), data['node', 'rev_to', 'node'].edge_attr], dim=1)
        else:
            data.edge_attr = torch.cat([torch.arange(data.edge_attr.shape[0]).view(-1, 1), data.edge_attr], dim=1)


In [34]:


def get_loaders(tr_data, val_data, te_data,
                tr_inds, val_inds, te_inds,
                transform,
                num_neighs,
                batch_size,
                ):
    if isinstance(tr_data, HeteroData):
        tr_edge_label_index = tr_data['node', 'to', 'node'].edge_index
        tr_edge_label = tr_data['node', 'to', 'node'].y


        tr_loader =  LinkNeighborLoader(tr_data, num_neighbors=num_neighs,
                                    edge_label_index=(('node', 'to', 'node'), tr_edge_label_index),
                                    edge_label=tr_edge_label, batch_size=batch_size, shuffle=True, transform=transform)

        val_edge_label_index = val_data['node', 'to', 'node'].edge_index#[:,val_inds]
        val_edge_label = val_data['node', 'to', 'node'].y#[val_inds]


        val_loader =  LinkNeighborLoader(val_data, num_neighbors=num_neighs,
                                    edge_label_index=(('node', 'to', 'node'), val_edge_label_index),
                                    edge_label=val_edge_label, batch_size=batch_size, shuffle=False, transform=transform)

        te_edge_label_index = te_data['node', 'to', 'node'].edge_index#[:,te_inds]
        te_edge_label = te_data['node', 'to', 'node'].y#[te_inds]


        te_loader =  LinkNeighborLoader(te_data, num_neighbors=num_neighs,
                                    edge_label_index=(('node', 'to', 'node'), te_edge_label_index),
                                    edge_label=te_edge_label, batch_size=batch_size, shuffle=False, transform=transform)
    else:
        tr_loader =  LinkNeighborLoader(tr_data, num_neighbors=num_neighs, batch_size=batch_size, shuffle=True, transform=transform)
        val_loader = LinkNeighborLoader(val_data,num_neighbors=num_neighs, edge_label_index=val_data.edge_index,
                                        edge_label=val_data.y, batch_size=batch_size, shuffle=False, transform=transform) # y[val_inds] , edge_index[:, val_inds],
        te_loader =  LinkNeighborLoader(te_data,num_neighbors=num_neighs, edge_label_index=te_data.edge_index,
                                edge_label=te_data.y, batch_size=batch_size, shuffle=False, transform=transform)# y[te_inds] , edge_index[:, te_inds],

    return tr_loader, val_loader, te_loader


In [35]:
@torch.no_grad()
def evaluate_homo(loader, inds, model, data, device, tqdm, args_data):
    '''Evaluates the model performane for homogenous graph data.'''
    preds = []
    ground_truths = []
    for batch in tqdm.tqdm(loader, disable=not tqdm):
        #select the seed edges from which the batch was created
        # inds = inds.detach().cpu() # This is not needed
        # batch_edge_inds = inds[batch.input_id.detach().cpu()] # This is incorrect
        # batch_edge_ids = loader.data.edge_attr.detach().cpu()[batch_edge_inds, 0] # This is incorrect
        batch_edge_ids = loader.data.edge_attr.detach().cpu()[batch.input_id.detach().cpu(), 0] # Corrected indexing
        mask = torch.isin(batch.edge_attr[:, 0].detach().cpu(), batch_edge_ids)

        #add the seed edges that have not been sampled to the batch
        missing = ~torch.isin(batch_edge_ids, batch.edge_attr[:, 0].detach().cpu())

        if missing.sum() != 0 and (args_data == 'Small_J' or args_data == 'Small_Q'):
            missing_ids = batch_edge_ids[missing].int()
            n_ids = batch.n_id
            add_edge_index = data.edge_index[:, missing_ids].detach().clone()
            node_mapping = {value.item(): idx for idx, value in enumerate(n_ids)}
            add_edge_index = torch.tensor([[node_mapping[val.item()] for val in row] for row in add_edge_index])
            add_edge_attr = data.edge_attr[missing_ids, :].detach().clone()
            add_y = data.y[missing_ids].detach().clone()

            batch.edge_index = torch.cat((batch.edge_index, add_edge_index), 1)
            batch.edge_attr = torch.cat((batch.edge_attr, add_edge_attr), 0)
            batch.y = torch.cat((batch.y, add_y), 0)

            mask = torch.cat((mask, torch.ones(add_y.shape[0], dtype=torch.bool)))

        #remove the unique edge id from the edge features, as it's no longer needed
        batch.edge_attr = batch.edge_attr[:, 1:]

        with torch.no_grad():
            batch.to(device)
            out = model(batch.x, batch.edge_index, batch.edge_attr)
            out = out[mask]
            pred = out.argmax(dim=-1)
            preds.append(pred)
            ground_truths.append(batch.y[mask])
    pred = torch.cat(preds, dim=0).cpu().numpy()
    ground_truth = torch.cat(ground_truths, dim=0).cpu().numpy()
    f1 = f1_score(ground_truth, pred)

    return f1

In [36]:
@torch.no_grad()
def evaluate_hetero(loader, inds, model, data, device, tqdm, args_data):
    '''Evaluates the model performane for heterogenous graph data.'''
    preds = []
    ground_truths = []
    for batch in tqdm.tqdm(loader, disable=not tqdm):
        #select the seed edges from which the batch was created
        # inds = inds.detach().cpu() # This is not needed
        batch_edge_inds = batch['node', 'to', 'node'].input_id.detach().cpu() # Use input_id directly
        batch_edge_ids = loader.data['node', 'to', 'node'].edge_attr.detach().cpu()[batch_edge_inds, 0]
        mask = torch.isin(batch['node', 'to', 'node'].edge_attr[:, 0].detach().cpu(), batch_edge_ids)

        #add the seed edges that have not been sampled to the batch
        missing = ~torch.isin(batch_edge_ids, batch['node', 'to', 'node'].edge_attr[:, 0].detach().cpu())

        if missing.sum() != 0 and (args_data == 'Small_J' or args_data == 'Small_Q'):
            missing_ids = batch_edge_ids[missing].int()
            n_ids = batch['node'].n_id
            add_edge_index = data['node', 'to', 'node'].edge_index[:, missing_ids].detach().clone()
            node_mapping = {value.item(): idx for idx, value in enumerate(n_ids)}
            add_edge_index = torch.tensor([[node_mapping[val.item()] for val in row] for row in add_edge_index])
            add_edge_attr = data['node', 'to', 'node'].edge_attr[missing_ids, :].detach().clone()
            add_y = data['node', 'to', 'node'].y[missing_ids].detach().clone()

            batch['node', 'to', 'node'].edge_index = torch.cat((batch['node', 'to', 'node'].edge_index, add_edge_index), 1)
            batch['node', 'to', 'node'].edge_attr = torch.cat((batch['node', 'to', 'node'].edge_attr, add_edge_attr), 0)
            batch['node', 'to', 'node'].y = torch.cat((batch['node', 'to', 'node'].y, add_y), 0)

            mask = torch.cat((mask, torch.ones(add_y.shape[0], dtype=torch.bool)))

        #remove the unique edge id from the edge features, as it's no longer needed
        batch['node', 'to', 'node'].edge_attr = batch['node', 'to', 'node'].edge_attr[:, 1:]
        batch['node', 'rev_to', 'node'].edge_attr = batch['node', 'rev_to', 'node'].edge_attr[:, 1:]

        with torch.no_grad():
            batch.to(device)
            out = model(batch.x_dict, batch.edge_index_dict, batch.edge_attr_dict)
            out = out[('node', 'to', 'node')]
            out = out[mask]
            pred = out.argmax(dim=-1)
            preds.append(pred)
            ground_truths.append(batch['node', 'to', 'node'].y[mask])
    pred = torch.cat(preds, dim=0).cpu().numpy()
    ground_truth = torch.cat(ground_truths, dim=0).cpu().numpy()
    f1 = f1_score(ground_truth, pred)

    return f1

In [37]:
def save_model(model, optimizer, epoch, unique_name, finetune, model_save_path):
    # Save the model in a dictionary
    torch.save({
                'epoch': epoch + 1,
                'model_state_dict': model.state_dict(),
                'optimizer_state_dict': optimizer.state_dict()
                }, f'{model_save_path}/checkpoint_{unique_name}{"" if not finetune else "_finetuned"}.tar')

def load_model(model, device, unique_name, config, model_load_path):
    checkpoint = torch.load(f'{model_load_path}/checkpoint_{unique_name}.tar')
    model.load_state_dict(checkpoint['model_state_dict'])
    model.to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=config.lr)
    optimizer.load_state_dict(checkpoint['optimizer_state_dict'])

    return model, optimizer

## Training Handler

In [38]:
# import torch
# import tqdm
# from sklearn.metrics import f1_score
# # from train_util import AddEgoIds, extract_param, add_arange_ids, get_loaders, evaluate_homo, evaluate_hetero, save_model, load_model
# # from models import GINe, PNA, GATe, RGCN
# from torch_geometric.data import Data, HeteroData
# from torch_geometric.nn import to_hetero, summary
# from torch_geometric.utils import degree
# import wandb
# # import logging

In [39]:
def get_model(sample_batch,
              config,
              model : str,
              emlps,
              ):
    n_feats = sample_batch.x.shape[1] if not isinstance(sample_batch, HeteroData) else sample_batch['node'].x.shape[1]
    e_dim = (sample_batch.edge_attr.shape[1] - 1) if not isinstance(sample_batch, HeteroData) else (sample_batch['node', 'to', 'node'].edge_attr.shape[1] - 1)

    if model == "gin":
        model = GINe(
                num_features=n_feats, num_gnn_layers=config.n_gnn_layers, n_classes=2,
                n_hidden=round(config.n_hidden), residual=False, edge_updates=emlps, edge_dim=e_dim,
                dropout=config.dropout, final_dropout=config.final_dropout
                )
    elif model == "gat":
        model = GATe(
                num_features=n_feats, num_gnn_layers=config.n_gnn_layers, n_classes=2,
                n_hidden=round(config.n_hidden), n_heads=round(config.n_heads),
                edge_updates=emlps, edge_dim=e_dim,
                dropout=config.dropout, final_dropout=config.final_dropout
                )
    elif model == "pna":
        if not isinstance(sample_batch, HeteroData):
            d = degree(sample_batch.edge_index[1], dtype=torch.long)
        else:
            index = torch.cat((sample_batch['node', 'to', 'node'].edge_index[1], sample_batch['node', 'rev_to', 'node'].edge_index[1]), 0)
            d = degree(index, dtype=torch.long)
        deg = torch.bincount(d, minlength=1)
        model = PNA(
            num_features=n_feats, num_gnn_layers=config.n_gnn_layers, n_classes=2,
            n_hidden=round(config.n_hidden), edge_updates=emlps, edge_dim=e_dim,
            dropout=config.dropout, deg=deg, final_dropout=config.final_dropout
            )
    elif config.model == "rgcn":
        model = RGCN(
            num_features=n_feats, edge_dim=e_dim, num_relations=8, num_gnn_layers=round(config.n_gnn_layers),
            n_classes=2, n_hidden=round(config.n_hidden),
            edge_update=emlps, dropout=config.dropout, final_dropout=config.final_dropout, n_bases=None #(maybe)
        )

    return model


In [40]:
def train_homo(tr_loader, val_loader, te_loader,
               tr_inds, val_inds, te_inds,
               model, optimizer,
               loss_fn,
               args_tqdm ,
               args_data : str,
               unique_name : str,
               finetune ,
               model_save,
               config, device,
               val_data, te_data,
               model_save_path):
    #training
    best_val_f1 = 0
    for epoch in range(config.epochs):
        total_loss = total_examples = 0
        preds = []
        ground_truths = []
        for batch in tqdm.tqdm(tr_loader, disable=not args_tqdm):
            optimizer.zero_grad()
            #select the seed edges from which the batch was created
            batch_edge_ids = tr_loader.data.edge_attr.detach().cpu()[batch.input_id.detach().cpu(), 0]

            mask = torch.isin(batch.edge_attr[:, 0].detach().cpu(), batch_edge_ids)

            #remove the unique edge id from the edge features, as it's no longer needed
            batch.edge_attr = batch.edge_attr[:, 1:]

            batch.to(device)
            out = model(batch.x, batch.edge_index, batch.edge_attr)
            pred = out[mask]
            ground_truth = batch.y[mask]
            preds.append(pred.argmax(dim=-1))
            ground_truths.append(ground_truth)
            loss = loss_fn(pred, ground_truth)

            loss.backward()
            optimizer.step()

            total_loss += float(loss) * pred.numel()
            total_examples += pred.numel()

        pred = torch.cat(preds, dim=0).detach().cpu().numpy()
        ground_truth = torch.cat(ground_truths, dim=0).detach().cpu().numpy()
        f1 = f1_score(ground_truth, pred)
        wandb.log({"f1/train": f1}, step=epoch)
        print(f'Train F1: {f1:.4f}') # logging.info

        #evaluate
        val_f1 = evaluate_homo(val_loader, val_inds, model, val_data, device, tqdm, args_data)
        te_f1 = evaluate_homo(te_loader, te_inds, model, te_data, device, tqdm, args_data)

        wandb.log({"f1/validation": val_f1}, step=epoch)
        wandb.log({"f1/test": te_f1}, step=epoch)
        print(f'Validation F1: {val_f1:.4f}')# logging.info
        print(f'Test F1: {te_f1:.4f}')# logging.info

        if epoch == 0:
            wandb.log({"best_test_f1": te_f1}, step=epoch)
        elif val_f1 > best_val_f1:
            best_val_f1 = val_f1
            wandb.log({"best_test_f1": te_f1}, step=epoch)
            if model_save:
                save_model(model, optimizer, epoch, unique_name, finetune, model_save_path)

    return model

In [41]:
def train_hetero(tr_loader, val_loader, te_loader,
                 tr_inds, val_inds, te_inds,
                 model, optimizer,
                 loss_fn,
                 args_tqdm,
                 args_data,
                 unique_name,
                 finetune,
                 model_save,
                 config, device,
                 val_data, te_data,
                 model_save_path):
    #training
    best_val_f1 = 0
    for epoch in range(config.epochs):
        total_loss = total_examples = 0
        preds = []
        ground_truths = []
        for batch in tqdm.tqdm(tr_loader, disable=not args_tqdm):
            optimizer.zero_grad()
            #select the seed edges from which the batch was created
            # inds = tr_inds.detach().cpu() # This is not needed
            # batch_edge_inds = batch['node', 'to', 'node'].input_id.detach().cpu() # Use input_id directly
            batch_edge_ids = tr_loader.data['node', 'to', 'node'].edge_attr.detach().cpu()[batch['node', 'to', 'node'].input_id.detach().cpu(), 0] # Corrected indexing
            mask = torch.isin(batch['node', 'to', 'node'].edge_attr[:, 0].detach().cpu(), batch_edge_ids)

            #remove the unique edge id from the edge features, as it's no longer needed
            batch['node', 'to', 'node'].edge_attr = batch['node', 'to', 'node'].edge_attr[:, 1:]
            batch['node', 'rev_to', 'node'].edge_attr = batch['node', 'rev_to', 'node'].edge_attr[:, 1:]

            batch.to(device)
            out = model(batch.x_dict, batch.edge_index_dict, batch.edge_attr_dict)
            out = out[('node', 'to', 'node')]
            pred = out[mask]
            ground_truth = batch['node', 'to', 'node'].y[mask]
            preds.append(pred.argmax(dim=-1))
            ground_truths.append(batch['node', 'to', 'node'].y[mask])
            loss = loss_fn(pred, ground_truth)

            loss.backward()
            optimizer.step()

            total_loss += float(loss) * pred.numel()
            total_examples += pred.numel()

        pred = torch.cat(preds, dim=0).detach().cpu().numpy()
        ground_truth = torch.cat(ground_truths, dim=0).detach().cpu().numpy()
        f1 = f1_score(ground_truth, pred)
        wandb.log({"f1/train": f1}, step=epoch)
        print(f'Train F1: {f1:.4f}') # logging.info

        #evaluate
        val_f1 = evaluate_hetero(val_loader, val_inds, model, val_data, device, tqdm, args_data)
        te_f1 = evaluate_hetero(te_loader, te_inds, model, te_data, device, tqdm, args_data)

        wandb.log({"f1/validation": val_f1}, step=epoch)
        wandb.log({"f1/test": te_f1}, step=epoch)
        print(f'Validation F1: {val_f1:.4f}')# logging.info
        print(f'Test F1: {te_f1:.4f}')# logging.info

        if epoch == 0:
            wandb.log({"best_test_f1": te_f1}, step=epoch)
        elif val_f1 > best_val_f1:
            best_val_f1 = val_f1
            wandb.log({"best_test_f1": te_f1}, step=epoch)
            if model_save:
                save_model(model, optimizer, epoch, unique_name, finetune, model_save_path)

    return model

In [42]:
def train_gnn(tr_data, val_data, te_data,
              tr_inds, val_inds, te_inds,
              tqdm,
              testing,
              n_epochs,
              batch_size,
              model_name : str,
              unique_name,
              args_data,
              num_neighs,
              ego,
              reverse_mp,
              finetune,
              emlps,
              model_save,
              model_load_path,
              model_save_path,
              ):
    #set device
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

    #define a model config dictionary and wandb logging at the same time
    wandb.init(
        mode="disabled" if testing else "online",
        project="Multi_GAT_3", #replace this with your wandb project name if you want to use wandb logging

        config={
            "epochs": n_epochs,
            "batch_size": batch_size,
            "model": model_name,
            "data": args_data,
            "num_neighbors": num_neighs,
            "lr": extract_param("lr", model_name),
            "n_hidden": extract_param("n_hidden", model_name),
            "n_gnn_layers": extract_param("n_gnn_layers", model_name),
            "loss": "ce",
            "w_ce1": extract_param("w_ce1", model_name),
            "w_ce2": extract_param("w_ce2", model_name),
            "dropout": extract_param("dropout", model_name),
            "final_dropout": extract_param("final_dropout", model_name),
            "n_heads": extract_param("n_heads", model_name) if model_name == 'gat' else None
        }
    )

    config = wandb.config

    #set the transform if ego ids should be used
    if ego:
        transform = AddEgoIds()
    else:
        transform = None

    #add the unique ids to later find the seed edges
    add_arange_ids([tr_data, val_data, te_data])

    tr_loader, val_loader, te_loader = get_loaders(tr_data, val_data, te_data, tr_inds, val_inds, te_inds, transform, num_neighs, batch_size)

    #get the model
    sample_batch = next(iter(tr_loader))
    model = get_model(sample_batch, config, model_name, emlps)

    if reverse_mp:
        model = to_hetero(model, te_data.metadata(), aggr='mean')

    if finetune:
        model, optimizer = load_model(model, device, unique_name, config, model_load_path)
    else:
        model.to(device)
        optimizer = torch.optim.Adam(model.parameters(), lr=config.lr)

    sample_batch.to(device)
    sample_x = sample_batch.x if not isinstance(sample_batch, HeteroData) else sample_batch.x_dict
    sample_edge_index = sample_batch.edge_index if not isinstance(sample_batch, HeteroData) else sample_batch.edge_index_dict
    if isinstance(sample_batch, HeteroData):
        sample_batch['node', 'to', 'node'].edge_attr = sample_batch['node', 'to', 'node'].edge_attr[:, 1:]
        sample_batch['node', 'rev_to', 'node'].edge_attr = sample_batch['node', 'rev_to', 'node'].edge_attr[:, 1:]
    else:
        sample_batch.edge_attr = sample_batch.edge_attr[:, 1:]
    sample_edge_attr = sample_batch.edge_attr if not isinstance(sample_batch, HeteroData) else sample_batch.edge_attr_dict
    print(summary(model, sample_x, sample_edge_index, sample_edge_attr)) # logging.info

    loss_fn = torch.nn.CrossEntropyLoss(weight=torch.FloatTensor([config.w_ce1, config.w_ce2]).to(device))

    if reverse_mp:
        model = train_hetero(tr_loader, val_loader, te_loader,
                             tr_inds, val_inds, te_inds,
                             model, optimizer,
                             loss_fn,
                             tqdm,
                             args_data,
                             unique_name,
                             finetune,
                             model_save,
                             config,
                             device,
                             val_data, te_data,
                             model_save_path)
    else:
        model = train_homo(tr_loader, val_loader, te_loader,
                           tr_inds, val_inds, te_inds,
                           model, optimizer,
                           loss_fn,
                           tqdm,
                           args_data,
                           unique_name,
                           finetune,
                           model_save,
                           config, device,
                           val_data, te_data,
                           model_save_path)

    wandb.finish()

## Inference Functions

In [43]:
# import torch
# import pandas as pd
# # from train_util import AddEgoIds, extract_param, add_arange_ids, get_loaders, evaluate_homo, evaluate_hetero
# # from training import get_model
# from torch_geometric.nn import to_hetero, summary
# import wandb
# import logging
# import os
# import sys
# import time

In [44]:
# script_start = time.time()

def infer_gnn(tr_data, val_data, te_data,
              tr_inds, val_inds, te_inds,
              testing,
              n_epochs,
              batch_size,
              model_name : str,
              unique_name,
              args_data,
              num_neighs,
              ego,
              reverse_mp,
              finetune,
              emlps,
              model_save,
              model_load_path,
              model_save_path,):
    #set device
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

    #define a model config dictionary and wandb logging at the same time
    wandb.init(
        mode="disabled" if testing else "online",
        project="Anti_money_laundering",

        config={
            "epochs": n_epochs,
            "batch_size": batch_size,
            "model": model_name,
            "data": args_data,
            "num_neighbors": num_neighs,
            "lr": extract_param("lr", model_name),
            "n_hidden": extract_param("n_hidden", model_name),
            "n_gnn_layers": extract_param("n_gnn_layers", model_name),
            "loss": "ce",
            "w_ce1": extract_param("w_ce1", model_name),
            "w_ce2": extract_param("w_ce2", model_name),
            "dropout": extract_param("dropout", model_name),
            "final_dropout": extract_param("final_dropout", model_name),
            "n_heads": extract_param("n_heads", model_name) if model_name == 'gat' else None
        }
    )

    config = wandb.config

    #set the transform if ego ids should be used
    if ego:
        transform = AddEgoIds()
    else:
        transform = None

    #add the unique ids to later find the seed edges
    add_arange_ids([tr_data, val_data, te_data])

    tr_loader, val_loader, te_loader = get_loaders(tr_data, val_data, te_data,
                                                   tr_inds, val_inds, te_inds,
                                                   transform,
                                                   num_neighs, batch_size,)

    #get the model
    sample_batch = next(iter(tr_loader))
    model = get_model(sample_batch, config, model_name, emlps)

    if reverse_mp:
        model = to_hetero(model, te_data.metadata(), aggr='mean')

    # if not (avg_tps or finetune):
    #     command = " ".join(sys.argv)
    #     name = ""
    #     name = '-'.join(name.split('-')[3:])
    #     unique_name = name

    print("=> loading model checkpoint")# logging.info
    checkpoint = torch.load(f'{model_load_path}/checkpoint_{unique_name}.tar')
    start_epoch = checkpoint['epoch']
    model.load_state_dict(checkpoint['model_state_dict'])
    model.to(device)

    print("=> loaded checkpoint (epoch {})".format(start_epoch)) # logging.info

    if not reverse_mp:
        te_f1 = evaluate_homo(te_loader, te_inds,
                                               model,
                                               te_data,
                                               device,
                                               tqdm,
                                               args_data,
                                               )
    else:
        te_f1 = evaluate_hetero(te_loader, te_inds,
                                                 model,
                                                 te_data,
                                                 device,
                                                 tqdm,
                                                 args_data,
                                                 )
    wandb.log({"f1/GAT2" : te_f1} , step = start_epoch)
    print(f'Test F1: {te_f1:.4f}')# logging.info

    wandb.finish()

 # Training

## utils

In [45]:
# # import argparse
# import numpy as np
# import torch
# import random
# # import logging
# import os
# import sys

In [46]:
# def create_parser():
#     parser = argparse.ArgumentParser()

#     #Adaptations
#     parser.add_argument("--emlps", action='store_true', help="Use emlps in GNN training")
#     parser.add_argument("--reverse_mp", action='store_true', help="Use reverse MP in GNN training")
#     parser.add_argument("--ports", action='store_true', help="Use port numberings in GNN training")
#     parser.add_argument("--tds", action='store_true', help="Use time deltas (i.e. the time between subsequent transactions) in GNN training")
#     parser.add_argument("--ego", action='store_true', help="Use ego IDs in GNN training")

#     #Model parameters
#     parser.add_argument("--batch_size", default=8192, type=int, help="Select the batch size for GNN training")
#     parser.add_argument("--n_epochs", default=100, type=int, help="Select the number of epochs for GNN training")
#     parser.add_argument('--num_neighs', nargs='+', default=[100,100], help='Pass the number of neighors to be sampled in each hop (descending).')

#     #Misc
#     parser.add_argument("--seed", default=1, type=int, help="Select the random seed for reproducability")
#     parser.add_argument("--tqdm", action='store_true', help="Use tqdm logging (when running interactively in terminal)")
#     parser.add_argument("--data", default=None, type=str, help="Select the AML dataset. Needs to be either small or medium.", required=True)
#     parser.add_argument("--model", default=None, type=str, help="Select the model architecture. Needs to be one of [gin, gat, rgcn, pna]", required=True)
#     parser.add_argument("--testing", action='store_true', help="Disable wandb logging while running the script in 'testing' mode.")
#     parser.add_argument("--save_model", action='store_true', help="Save the best model.")
#     parser.add_argument("--unique_name", action='store_true', help="Unique name under which the model will be stored.")
#     parser.add_argument("--finetune", action='store_true', help="Fine-tune a model. Note that args.unique_name needs to point to the pre-trained model.")
#     parser.add_argument("--inference", action='store_true', help="Load a trained model and only do AML inference with it. args.unique name needs to point to the trained model.")

#     return parser

In [47]:
# def logger_setup():
#     # Setup logging
#     log_directory = "logs"
#     if not os.path.exists(log_directory):
#         os.makedirs(log_directory)
#     logging.basicConfig(
#         level=logging.INFO,
#         format="%(asctime)s [%(levelname)-5.5s] %(message)s",
#         handlers=[
#             logging.FileHandler(os.path.join(log_directory, "logs.log")),     ## log to local log file
#             logging.StreamHandler(sys.stdout)          ## log also to stdout (i.e., print to screen)
#         ]
#     )


In [48]:
def set_seed(seed: int = 0) -> None:
    np.random.seed(seed)
    random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    # When running on the CuDNN backend, two further options must be set
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    # Set a fixed value for the hash seed
    os.environ["PYTHONHASHSEED"] = str(seed)
    logging.info(f"Random seed set as {seed}")

## : _ :

In [49]:
# import time
# import logging
# # from util import create_parser, set_seed, logger_setup
# # from data_loading import get_data
# # from training import train_gnn
# # from inference import infer_gnn
# import json

In [50]:
# args
    # parser = create_parser()
    # args = parser.parse_args()

args_tqdm = True
emlps = True
reverse_mp = True
port = True
tds = False
ego = True
batch_size = 8192
n_epochs = 11
num_neighs = [100, 100]
seed = 11
testing = False
model_save = True
unique_name = 'Multi_GAT_2'
finetune = False
inference = True
args_data = "small"
model_name = "gat"

In [51]:
    # with open('data_config.json', 'r') as config_file:
    #     data_config = json.load(config_file)

    #data_config variables

aml_data = data_path
model_load_path = model_load_path
model_save_path = model_save_path
formatted_data_path = data_path + f"/{args_data}/formatted_transactions.csv"

In [52]:
# Setup logging
# logger_setup()

In [53]:
#set seed
set_seed(seed)

## prepared_data


In [54]:
if computation=='local' :
  prepared_data_path = '../data/small/prepared_data_Multi_GIN'
else :
  prepared_data_path = '/content/drive/MyDrive/uco_fraud_detector/data/AML/small/prepared_data_Multi_GIN'

### Memoizing Prepared_data

In [55]:
# prepared_data = {
#     'tr_data': tr_data,
#     'val_data': val_data,
#     'te_data': te_data,
#     'tr_inds': tr_inds,
#     'val_inds': val_inds,
#     'te_inds': te_inds,
#     'reverse_mp': reverse_mp,
#     'emlps': emlps,
#     'port': port,
#     'tds': tds,
#     'ego': ego,
# }

In [56]:
# import pickle
# pickle.dump(prepared_data, open(prepared_data_path, 'wb'))

### Extracting prepared_data

In [57]:
import pickle
prepared_data = pickle.load(open(prepared_data_path, 'rb'))
print(prepared_data.keys())

dict_keys(['tr_data', 'val_data', 'te_data', 'tr_inds', 'val_inds', 'te_inds', 'reverse_mp', 'emlps', 'tds', 'port', 'ego'])


In [58]:
tr_data = prepared_data['tr_data']
val_data = prepared_data['val_data']
te_data = prepared_data['te_data']
tr_inds = prepared_data['tr_inds']
val_inds = prepared_data['val_inds']
te_inds = prepared_data['te_inds']

In [59]:
print('tr_data : \n',tr_data,'\n')
print('val_data : \n',val_data,'\n')
print('te_data : \n',te_data,'\n')
print('tr_inds : \n',tr_inds,'\n')
print('val_inds : \n',val_inds,'\n')
print('te_inds : \n',te_inds,'\n')

tr_data : 
 HeteroGraphData(
  readout='edge',
  node={ x=[515088, 1] },
  (node, to, node)={
    edge_index=[2, 3248921],
    edge_attr=[3248921, 6],
    y=[3248921],
    timestamps=[3248921],
  },
  (node, rev_to, node)={
    edge_index=[2, 3248921],
    edge_attr=[3248921, 6],
  }
) 

val_data : 
 HeteroGraphData(
  readout='edge',
  node={ x=[515088, 1] },
  (node, to, node)={
    edge_index=[2, 4214445],
    edge_attr=[4214445, 6],
    y=[4214445],
    timestamps=[4214445],
  },
  (node, rev_to, node)={
    edge_index=[2, 4214445],
    edge_attr=[4214445, 6],
  }
) 

te_data : 
 HeteroGraphData(
  readout='edge',
  node={ x=[515088, 1] },
  (node, to, node)={
    edge_index=[2, 5078345],
    edge_attr=[5078345, 6],
    y=[5078345],
    timestamps=[5078345],
  },
  (node, rev_to, node)={
    edge_index=[2, 5078345],
    edge_attr=[5078345, 6],
  }
) 

tr_inds : 
 tensor([      0,       1,       2,  ..., 3249981, 3249982, 3249983]) 

val_inds : 
 tensor([ 695463,  695464,  695465,  

In [60]:
torch.cuda.is_available()

True

## ..

In [61]:
# print("Retrieving data") #logging.info
# t1 = time.perf_counter()

# tr_data, val_data, te_data, tr_inds, val_inds, te_inds = get_data(model_name,
#                                                                   formatted_data_path,
#                                                                   port,
#                                                                   tds,
#                                                                   reverse_mp,
#                                                                   )

# t2 = time.perf_counter()
# print(f"Retrieved data in {t2-t1:.2f}s \n")#logging.info

In [62]:
if not inference:
    #Training
    print(f"Running Training") #logging.info
    train_gnn(tr_data, val_data, te_data,
              tr_inds, val_inds, te_inds,
              args_tqdm,
              testing,
              n_epochs,
              batch_size,
              model_name,
              unique_name,
              args_data,
              num_neighs,
              ego,
              reverse_mp,
              finetune,
              emlps,
              model_save,
              model_load_path,
              model_save_path,)
else:
    #Inference
    print(f"Running Inference")#logging.info
    infer_gnn(tr_data, val_data, te_data,
              tr_inds, val_inds, te_inds,
              testing,
              n_epochs,
              batch_size,
              model_name,
              unique_name,
              args_data,
              num_neighs,
              ego,
              reverse_mp,
              finetune,
              emlps,
              model_save,
              model_load_path,
              model_save_path)

Running Inference


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: sayak2103 (sayak2103-iiest-shibpur) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


/usr/local/lib/python3.11/dist-packages/torch_geometric/sampler/neighbor_sampler.py:61: UserWarning: Using 'NeighborSampler' without a 'pyg-lib' installation is deprecated and will be removed soon. Please install 'pyg-lib' for accelerated neighborhood sampling
  warnings.warn(f"Using '{self.__class__.__name__}' without a "


=> loading model checkpoint
=> loaded checkpoint (epoch 9)


100%|██████████| 620/620 [09:52<00:00,  1.05it/s]


Test F1: 0.1836


f1/GAT2,▁
f1/GAT2,0.18364


In [ ]:
# wandb_key = '829bf058a06bc7d70a24dac55a8e2a3e981f59c1'

## Testing Model

In [ ]:
# import networkx as nx
# import matplotlib.pyplot as plt

# # Assuming 'te_data' is your GraphData object containing the entire graph
# # You might need to adjust this based on which data split you want to visualize

# # Create a NetworkX graph
# G = nx.Graph()

# # Add nodes
# # The number of nodes is inferred from the maximum node index in edge_index
# num_nodes = te_data.edge_index.max().item() + 1
# G.add_nodes_from(range(num_nodes))

# # Add edges with attributes
# # Assuming the edge_index is in the format [from_nodes, to_nodes]
# from_nodes = te_data.edge_index[0].tolist()
# to_nodes = te_data.edge_index[1].tolist()
# edge_attributes = te_data.edge_attr.tolist() # Assuming edge_attr is a tensor

# for i in range(len(from_nodes)):
#     # You can add edge attributes here if needed, e.g., weight=edge_attributes[i][some_attribute_index]
#     G.add_edge(from_nodes[i], to_nodes[i])

# # Define node colors based on the 'y' attribute
# # Assuming 'y' is an edge attribute in te_data.edge_attr and indicates the classification
# # You need to know which column in edge_attr corresponds to 'y'
# # For example, if 'Is Laundering' was the last column before adding the unique id (which is now the first),
# # and you removed the unique id in the evaluation function, the 'y' classification might be something else now.
# # Let's assume for now that the classification is in the last column of the original edge_attr before any additions or removals.
# # You might need to adjust the index below based on your actual data structure after preprocessing.

# # Find the original 'Is Laundering' column index before adding the unique ID
# # Looking at the data loading, 'Is Laundering' is the last column of the original 'formatted' dataframe.
# # In get_data, edge_attr is created from ['Timestamp', 'Amount Received', 'Received Currency', 'Payment Format']
# # and then possibly 'ports' and 'time_deltas' are added.
# # 'y' is created separately from 'Is Laundering'.
# # So, 'y' is a separate tensor for each data split, not an edge attribute.

# # Let's assume you want to color the nodes based on whether they are involved in any laundering transaction.
# # We can get the nodes involved in laundering transactions from the edge_index where the corresponding y is 1.
# laundering_edge_indices = torch.where(te_data.y == 1)[0]
# laundering_nodes = torch.unique(te_data.edge_index[:, laundering_edge_indices].view(-1)).tolist()

# node_colors = ['red' if node in laundering_nodes else 'blue' for node in G.nodes()]


# # Draw the graph
# plt.figure(figsize=(12, 12))
# nx.draw(G, with_labels=False, node_size=10, node_color=node_colors, width=0.5)
# plt.title("Graph Visualization colored by involvement in laundering")
# plt.show()